### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load (Remember to Change These)
load_file = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purch_df = pd.read_csv(load_file)

## Player Count

* Display the total number of players


In [2]:
# Find number of unique players
num_players = purch_df[["SN"]].nunique()
# Create new dataframe
player_df = pd.DataFrame({"Total Players":num_players})
# Reset index to get rid of "SN" as the index
player_count = player_df.reset_index(drop=True)
player_count


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# Pull data for each column
unique_items = purch_df["Item ID"].nunique()
avg_price = round(purch_df["Price"].mean(),2)
num_purch = purch_df["Purchase ID"].count()
total_rev = purch_df["Price"].sum()
# Format avg price & total revenue as currency
avg_price_format = '$ {:,.2f}'.format(avg_price)
total_rev_format = '$ {:,.2f}'.format(total_rev)
# New dataframe to store calculations
purch_analysis_summary = pd.DataFrame({"Unique Items Purchased": unique_items,"Average Price":[avg_price_format],
                           "Purchase Total": num_purch,"Total Revenue": total_rev_format})

purch_analysis_summary

,Unique Items Purchased,Average Price,Purchase Total,Total Revenue
0,179,$ 3.05,780,"$ 2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# Pull in SN & Gender to find unique players & their gender
ply_gender = purch_df[["SN","Gender"]]
# Clean data by deleting duplicates
del_duplicates = ply_gender.drop_duplicates(subset="SN")
# Calculate gender individual percentages of total & format as percent
percentOf = round((del_duplicates["Gender"].value_counts()/del_duplicates["Gender"].count())*100.0,2).astype(str)+"%"
# Create dataframe for gender & percentage of total
gender_dem = pd.DataFrame({"Gender":del_duplicates["Gender"].value_counts(),"Percent":percentOf})
gender_dem

,Gender,Percent
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# Created copy of DataFrame
purch_analysis = purch_df[["Gender","Price","SN"]].copy()
# Grouped data by specified column label
purch_group = purch_analysis.groupby(["Gender"])
# Calculate new columns using .agg to find count, average, total
price_calc = purch_group[["Gender","Price"]].agg(["count",np.average,np.sum])
# flattened multi-index hierarchy. This is to make column renaming easier later
price_calc.columns = price_calc.columns.get_level_values(1)
# Calculate avg total purchase price per person
avg_price_per_person = price_calc.iloc[:,2]/del_duplicates["Gender"].value_counts()
# Create new column for avg price per person
price_calc["Avg Total Purchase per Person"] = avg_price_per_person
# Rename columns 
purch_gender_summary = price_calc.rename(columns={"count":"Purchase Count",
                                                    "average":"Average Purchase Price",                           
                                                    "sum":"Total Purchase Value"})
# Format dataframe numbers as currency
purch_gender_summary["Average Purchase Price"] = purch_gender_summary["Average Purchase Price"].map("${:.2f}".format)
purch_gender_summary["Total Purchase Value"] = purch_gender_summary["Total Purchase Value"].map("${:.2f}".format)
purch_gender_summary["Avg Total Purchase per Person"] = purch_gender_summary["Avg Total Purchase per Person"].map("${:.2f}".format)
purch_gender_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# Establishe bins & labels
age_bin = [0,9,14.5,19.5,24.5,29.5,34.5,39.5,50]
age_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39",">40"]

In [7]:
# use pd.cut to bin values into discrete intervals
purch_df["Age Range"] = pd.cut(purch_df["Age"], age_bin, labels=age_labels)

In [8]:
# Group by new column "Age Group"
age_group = purch_df.groupby(["Age Range"])
# Find ages of players (used unique "SN" to find individual players)
age_group = age_group[["SN"]].nunique()
# Calculate percentage of players per category of age group
age_group["Percentage of Players"] = age_group.iloc[:,0]/age_group["SN"].agg(np.sum)
# Rename columns
age_dem_sum = age_group.rename(columns={"SN":"Total Count","Percentage of Players":"Percentage of Players"})
# Format dataframe "percent of players" as percent
age_dem_sum["Percentage of Players"] = age_dem_sum["Percentage of Players"].map("{:.2%}".format)
age_dem_sum

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
>40,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
# Created copy of DataFrame
purch_age = purch_df[["Age Range","Price"]].copy()
# Grouped data by specified column label
purch_age_group = purch_age.groupby(["Age Range"])
# Calculate new columns using .agg to find count, average, total
purch_age_price = purch_age_group[["Age Range","Price"]].agg(["count",np.average,np.sum])
# flattened multi-index hierarchy. This is to make column renaming easier later
purch_age_price.columns = purch_age_price.columns.get_level_values(1)
# Calculate avg total purchase price per person
age_avg_price_per_person = round((purch_age_price.iloc[:,2]/age_dem_sum.iloc[:,0]),2)
# Create new column for avg price per person
purch_age_price["Avg Total Purchase per Person"] = age_avg_price_per_person
# Renamed columns 
purch_age_summary = purch_age_price.rename(columns={"count":"Purchase Count",
                                                   "average":"Average Purchase Price",                           
                                                   "sum":"Total Purchase Value"})
# Format dataframe purchase data as currency 
purch_age_summary["Average Purchase Price"] = purch_age_summary["Average Purchase Price"].map("${:.2f}".format)
purch_age_summary["Total Purchase Value"] = purch_age_summary["Total Purchase Value"].map("${:.2f}".format)
purch_age_summary["Avg Total Purchase per Person"] = purch_age_summary["Avg Total Purchase per Person"].map("${:.2f}".format)
purch_age_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
>40,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
# Created copy of DataFrame
top_spend = purch_df[["SN","Price"]].copy()
# Grouped data by specified column label
top_spend_group = top_spend.groupby(["SN"])
# Calculate new columns using .agg to find count, average, total
top_spend_calc = top_spend_group[["SN","Price"]].agg(["count",np.average,np.sum])
# flattened multi-index hierarchy. This is to make column renaming easier later
top_spend_calc.columns = top_spend_calc.columns.get_level_values(1)
# Calculate avg total purchase price per person
spend_avg_price_per_person = round((purch_age_price.iloc[:,2]/age_dem_sum.iloc[:,0]),2)
# Create new column for avg price per person
top_spend_summary = top_spend_calc.rename(columns={"count":"Purchase Count",
                                                   "average":"Average Purchase Price",                           
                                                   "sum":"Total Purchase Value"})
# Sort columns on "Total Purchase Value" by decending order to obtain top spenders
top_spend_summary.sort_values(by=["Total Purchase Value"],ascending=False, inplace=True)
# Format data frame purchase data as currency
top_spend_summary["Average Purchase Price"] = top_spend_summary["Average Purchase Price"].map("${:.2f}".format)
top_spend_summary["Total Purchase Value"] = top_spend_summary["Total Purchase Value"].map("${:.2f}".format)
# preview dataframe
top_spend_summary.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
items = purch_df[["Item ID","Item Name", "Price"]].copy()
items_group = items.groupby(["Item ID","Item Name"])
# Calculate new columns using .agg to find count, average, total
items_group_calc = items_group[["Item Name","Price"]].agg(["count",np.average,np.sum])
# flattened multi-index hierarchy. This is to make column renaming easier later
items_group_calc.columns = items_group_calc.columns.get_level_values(1)
item_summary = items_group_calc.rename(columns={"count":"Purchase Count",
                                                   "average":"Average Item Price",                           
                                                   "sum":"Total Purchase Value"})
# Store dataframe without number formatting
item_summary_raw = item_summary
# Perform sort on most_popular data frame to obtain most popular items by purchase count
most_popular = item_summary_raw.sort_values(by=["Purchase Count"], ascending=False)
# Format most_popular  item price & purchase value data as currency
most_popular["Average Item Price"] = most_popular["Average Item Price"].map("${:.2f}".format)
most_popular["Total Purchase Value"] = most_popular["Total Purchase Value"].map("${:.2f}".format)
# preview dataframe
most_popular.head()

,,Purchase Count,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [12]:
# Access raw items_summary & perform sort on most_popular data frame to obtain most popular items by purchase value
most_profitable = item_summary_raw.sort_values(by=["Total Purchase Value"],ascending=False)
# Format most_profitable item price & purchase value data as currency
most_profitable["Average Item Price"] = most_profitable["Average Item Price"].map("${:.2f}".format)
most_profitable["Total Purchase Value"] = most_profitable["Total Purchase Value"].map("${:.2f}".format)
# preview dataframe
most_profitable.head()

,,Purchase Count,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
